In [1]:
from concrete.compiler import parameter
from pathlib import Path
from tempfile import TemporaryDirectory
from concrete.ml import deployment
import numpy as np
from concrete import fhe
from concrete.ml.deployment import FHEModelClient, FHEModelDev, FHEModelServer
from concrete.ml.sklearn import SGDClassifier
from deepface import DeepFace

/Users/ilyas/miniconda3/envs/ppai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [53]:
# Logistic regression sklearn
from sklearn.linear_model import LogisticRegression
from preprocess import load_dataset, featurisation
from sklearn.model_selection import train_test_split
embeddings, labels = load_dataset(
    "./data/lfw_people/George_HW_Bush", cache=True)
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42)

model = LogisticRegression(C=1/5)
model.fit(X_train, y_train)

score = model.score(X_test, y_test)
print(f"Accuracy: {score}")

Accuracy: 0.9893617021276596


In [59]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.linear_model import LogisticRegression

# Step 1: Train a simple sklearn model (e.g., Logistic Regression)
sklearn_model = LogisticRegression()
sklearn_model.fit(X_train, y_train)

class PyTorchLogisticRegression(nn.Module):
    def __init__(self, input_dim):
        super(PyTorchLogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
        
    def forward(self, x):
        return (self.linear(x) > 0).float()

# Step 3: Extract parameters from sklearn model
sklearn_weights = sklearn_model.coef_[0]
sklearn_bias = sklearn_model.intercept_[0]

# Step 4: Initialize and load parameters into PyTorch model
pytorch_model = PyTorchLogisticRegression(input_dim=128)
pytorch_model.linear.weight.data = torch.FloatTensor(sklearn_weights).unsqueeze(0)
pytorch_model.linear.bias.data = torch.FloatTensor([sklearn_bias])

sklearn_pred = sklearn_model.predict_proba(X_train)[:, 1]
pytorch_pred = pytorch_model(torch.FloatTensor(X_train)).detach().numpy().flatten()

print("Mean absolute difference in predictions:", np.mean(np.abs(sklearn_pred - pytorch_pred)))

Mean absolute difference in predictions: 0.0010453411443437591


In [60]:

X_train.shape, sklearn_weights.shape, sklearn_bias.shape

((375, 128), (128,), ())

In [61]:

from concrete.ml.torch.compile import compile_torch_model
import numpy

N_FEAT = 3
# Convert numpy arrays to torch tensors if they aren't already
torch_input = torch.from_numpy(X_train) if isinstance(X_train, np.ndarray) else X_train


quantized_module = compile_torch_model(
    pytorch_model, # our model
    torch_input, # a representative input-set to be used for both quantization and compilation
    n_bits=6,
    rounding_threshold_bits={"n_bits": 6, "method": "approximate"}
)

In [62]:
y_pred = quantized_module.forward(X_test, fhe="execute")

In [63]:
y_pred.ravel() - y_test

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.])